<a href="https://colab.research.google.com/github/JPUG26/proyecto_Kaggle_Modelos_I/blob/main/03%20-%20Modelo%20con%20preprocesado%20de%20agrupaci%C3%B3n%2C%20mapeo%20ordinal%20y%20OneHot%20y%20Radom%20Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03 - Modelo con preprocesado y Random forest


In [5]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

## Descargar archivo kaggle.json

Crear un archivo `kaggle.json` con su token de autenticación, subirlo a este espacio de trabajo del cuaderno y ejecutar la siguiente celda.

In [6]:
# Importar librería os (Operating System) de Python para modificar variables de entorno o gestionar archivos
import os

# Le indica a Kaggle dónde debe buscar el archivo de autenticación
os.environ['KAGGLE_CONFIG_DIR'] = '.'

# Comando de terminal linux para que solo el propietario pueda leer y escribir
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


## Descomprimir e inspeccionar datos

In [7]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [8]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


## Cargar datos de `train.csv` con pandas



In [9]:
import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
print ("shape of loaded dataframe", z.shape)


shape of loaded dataframe (692500, 21)


# **PREPROCESADO DE DATOS**

## Eliminar filas con muchos valores nulos

In [10]:
import numpy as np

cols = [
    "E_VALORMATRICULAUNIVERSIDAD","E_HORASSEMANATRABAJA","F_ESTRATOVIVIENDA",
    "F_TIENEINTERNET","F_EDUCACIONPADRE","F_TIENELAVADORA","F_TIENEAUTOMOVIL","E_PAGOMATRICULAPROPIO",
    "F_TIENECOMPUTADOR","F_EDUCACIONMADRE"
]

# Reemplazar indicadores comunes de "vacío" por NaN
z[cols] = z[cols].replace({'': np.nan, 'NA': np.nan, 'N/A': np.nan, '-': np.nan})

# Limpiar celdas que solo tengan espacios
z[cols] = z[cols].replace(r'^\s*$', np.nan, regex=True)

# Define columnas obligatorias y el umbral, se puso como obligatoria la columna E_VALORMATRICULAUNIVERSIDAD por la importancia que
# ha tenido en la mayor parte de los indicadores
col_obligatoria = "E_VALORMATRICULAUNIVERSIDAD"

# Obtenemos la lista de las otras 9 columnas "opcionales"
cols_opcionales = [col for col in cols if col != col_obligatoria]

# Se define cuántos valores NO NULOS deben existir en las 9 columnas opcionales
# para salvar una fila si la columna obligatoria (E_VALORMATRICULAUNIVERSIDAD) Es nula.

# Por ejemplo: Si se pone 5, significa: "Si la matrícula es nula,
#           conserva la fila si al menos 5 de las otras 9 columnas tiene un valor no nulo".
min_non_na_opcionales = 5

# --- Construcción de las condiciones de filtrado ---

# Condición 1: Conservar si la columna obligatoria NO es nula
condicion_1 = z[col_obligatoria].notnull()

# Condición 2: Conservar si la columna obligatoria sí es nula,
# Pero las columnas opcionales cumplen el umbral mínimo de valores no nulos.
condicion_2 = (z[col_obligatoria].isnull()) & \
              (z[cols_opcionales].notnull().sum(axis=1) >= min_non_na_opcionales)

print("Dimensiones del dataframe antes de la depuración")
print(z.shape)
# Conservamos las filas que cumplen la Condición 1 o la Condición 2
z = z[condicion_1 | condicion_2].reset_index(drop=True)

# Resultado
print("Dimensiones del dataframe depurado")
print(z.shape)

Dimensiones del dataframe antes de la depuración
(692500, 21)
Dimensiones del dataframe depurado
(690792, 21)


## Creación de nueva columna en el Dataframe que agrupa en áreas de conocimiento los diferentes programas de la columna E_PRGM_ACADEMICO

In [11]:
import pandas as pd
import unicodedata
import re

# Definición de Funciones de Limpieza y Mapeo

def corregir_codificacion_consorcio(texto):
    """
    Corrige errores comunes de codificación (Mojibake) donde
    UTF-8 se malinterpretó como latin-1 o windows-1252.
    Ej: 'ADMINISTRACIÃ“N' -> 'ADMINISTRACIÓN'
    """
    if not isinstance(texto, str):
        return texto
    try:
        # Intenta decodificar de latin-1 y recodificar a UTF-8
        return texto.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        # Si falla, devuelve el texto original
        return texto

def normalizar_texto(texto):
    """
    Limpia un string:
    1. Corrige la codificación
    2. Convierte a mayúsculas.
    3. Elimina acentos.
    """
    if not isinstance(texto, str):
        return ""

    # Paso 1: Corregir codificación
    texto = corregir_codificacion_consorcio(texto)

    # Paso 2: Convertir a mayúsculas
    texto = texto.upper()

    # Paso 3: Eliminar acentos (Normalización NFD)
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )

    return texto.strip()

# --- Diccionario de Palabras Clave y Áreas (Priorizado) ---
MAPEO_AREAS = {
    # --- 1. Educación ---
    "LICENCIATURA": "Educación (Licenciaturas)",
    "PEDAGOGIA": "Educación (Licenciaturas)",
    "ETNOEDUCACION": "Educación (Licenciaturas)",

    # --- 2. Ciencias de la Salud ---
    "ENFERMERIA": "Ciencias de la Salud",
    "MEDICINA": "Ciencias de la Salud",
    "ODONTOLOGIA": "Ciencias de la Salud",
    "PSICOLOGIA": "Ciencias de la Salud",
    "SALUD": "Ciencias de la Salud",
    "BACTERIOLOGIA": "Ciencias de la Salud",
    "FISIOTERAPIA": "Ciencias de la Salud",
    "NUTRICION": "Ciencias de la Salud",
    "INSTRUMENTACION QUIRURGICA": "Ciencias de la Salud",
    "OPTOMETRIA": "Ciencias de la Salud",
    "FONOAUDIOLOGIA": "Ciencias de la Salud",
    "TERAPIA": "Ciencias de la Salud",
    "VETERINARIA": "Ciencias de la Salud",
    "ZOOTECNIA": "Ciencias de la Salud",
    "BIOINGENIERIA": "Ciencias de la Salud",
    "BIOMEDICA": "Ciencias de la Salud",
    "GERONTOLOGIA": "Ciencias de la Salud",

    # --- 3. Ingeniería y Tecnología ---
    "INGENIERIA": "Ingeniería y Tecnología",
    "ARQUITECTURA": "Ingeniería y Tecnología",
    "SISTEMAS": "Ingeniería y Tecnología",
    "SOFTWARE": "Ingeniería y Tecnología",
    "COMPUTACION": "Ingeniería y Tecnología",
    "TELECOMUNICACIONES": "Ingeniería y Tecnología",
    "MECATRONICA": "Ingeniería y Tecnología",
    "ELECTRONICA": "Ingeniería y Tecnología",
    "MECANICA": "Ingeniería y Tecnología",
    "CIVIL": "Ingeniería y Tecnología",
    "TELEMATICA": "Ingeniería y Tecnología",
    "MULTIMEDIA": "Ingeniería y Tecnología",
    "URBANISMO": "Ingeniería y Tecnología",
    "CONSTRUCCION": "Ingeniería y Tecnología",
    "AERONAUTICA": "Ingeniería y Tecnología",
    "NANOTECNOLOGIA": "Ingeniería y Tecnología",
    "AUTOMATIZACION": "Ingeniería y Tecnología",

    # --- 4. Ciencias Económicas y Administrativas ---
    "ADMINISTRACION": "Ciencias Económicas y Administrativas",
    "ECONOMIA": "Ciencias Económicas y Administrativas",
    "CONTADURIA": "Ciencias Económicas y Administrativas",
    "FINANZAS": "Ciencias Económicas y Administrativas",
    "FINANCIERA": "Ciencias Económicas y Administrativas",
    "MERCADEO": "Ciencias Económicas y Administrativas",
    "PUBLICIDAD": "Ciencias Económicas y Administrativas",
    "NEGOCIOS": "Ciencias Económicas y Administrativas",
    "COMERCIO": "Ciencias Económicas y Administrativas",
    "HOTELERIA": "Ciencias Económicas y Administrativas",
    "TURISMO": "Ciencias Económicas y Administrativas",
    "LOGISTICA": "Ciencias Económicas y Administrativas",
    "BANCARIA": "Ciencias Económicas y Administrativas",
    "INDUSTRIAL": "Ciencias Económicas y Administrativas",

    # --- 5. Ciencias Sociales y Humanas ---
    "DERECHO": "Ciencias Sociales y Humanas",
    "JURISPRUDENCIA": "Ciencias Sociales y Humanas",
    "CIENCIA POLITICA": "Ciencias Sociales y Humanas",
    "TRABAJO SOCIAL": "Ciencias Sociales y Humanas",
    "SOCIOLOGIA": "Ciencias Sociales y Humanas",
    "ANTROPOLOGIA": "Ciencias Sociales y Humanas",
    "HISTORIA": "Ciencias Sociales y Humanas",
    "FILOSOFIA": "Ciencias Sociales y Humanas",
    "RELACIONES INTERNACIONALES": "Ciencias Sociales y Humanas",
    "GEOGRAFIA": "Ciencias Sociales y Humanas",
    "HUMANIDADES": "Ciencias Sociales y Humanas",
    "SOCIALES": "Ciencias Sociales y Humanas",
    "POLITICA": "Ciencias Sociales y Humanas",
    "GOBIERNO": "Ciencias Sociales y Humanas",
    "CRIMINALISTICA": "Ciencias Sociales y Humanas",
    "ARCHIVISTICA": "Ciencias Sociales y Humanas",
    "BIBLIOTECOLOGIA": "Ciencias Sociales y Humanas",
    "LENGUAS": "Ciencias Sociales y Humanas",
    "FILOLOGIA": "Ciencias Sociales y Humanas",
    "LETRAS": "Ciencias Sociales y Humanas",
    "LITERATURA": "Ciencias Sociales y Humanas",
    "LINGUISTICA": "Ciencias Sociales y Humanas",

    # --- 6. Artes, Diseño y Comunicación ---
    "DISEÑO": "Artes, Diseño y Comunicación",
    "COMUNICACION": "Artes, Diseño y Comunicación",
    "PERIODISMO": "Artes, Diseño y Comunicación",
    "ARTE": "Artes, Diseño y Comunicación",
    "ARTES": "Artes, Diseño y Comunicación",
    "MUSICA": "Artes, Diseño y Comunicación",
    "CINE": "Artes, Diseño y Comunicación",
    "AUDIOVISUAL": "Artes, Diseño y Comunicación",
    "FOTOGRAFIA": "Artes, Diseño y Comunicación",
    "DANZA": "Artes, Diseño y Comunicación",
    "TEATRO": "Artes, Diseño y Comunicación",
    "GASTRONOMIA": "Artes, Diseño y Comunicación",
    "CULINARIA": "Artes, Diseño y Comunicación",
    "MODAS": "Artes, Diseño y Comunicación",

    # --- 7. Ciencias Básicas y Naturales ---
    "BIOLOGIA": "Ciencias Básicas y Naturales",
    "QUIMICA": "Ciencias Básicas y Naturales",
    "FISICA": "Ciencias Básicas y Naturales",
    "MATEMATICAS": "Ciencias Básicas y Naturales",
    "ESTADISTICA": "Ciencias Básicas y Naturales",
    "GEOLOGIA": "Ciencias Básicas y Naturales",
    "ECOLOGIA": "Ciencias Básicas y Naturales",
    "ASTRONOMIA": "Ciencias Básicas y Naturales",
    "MICROBIOLOGIA": "Ciencias Básicas y Naturales",
    "GEOCIENCIAS": "Ciencias Básicas y Naturales",
    "BIOTECNOLOGIA": "Ciencias Básicas y Naturales",
    "BIOQUIMICA": "Ciencias Básicas y Naturales",

    # --- 8. Ciencias Agropecuarias y Ambientales ---
    "AGRONOMIA": "Ciencias Agropecuarias y Ambientales",
    "AGROPECUARIA": "Ciencias Agropecuarias y Ambientales",
    "AGROINDUSTRIAL": "Ciencias Agropecuarias y Ambientales",
    "AGRONEGOCIOS": "Ciencias Agropecuarias y Ambientales",
    "PECUARIA": "Ciencias Agropecuarias y Ambientales",
    "ACUICULTURA": "Ciencias Agropecuarias y Ambientales",
    "FORESTAL": "Ciencias Agropecuarias y Ambientales",
    "AGRICOLA": "Ciencias Agropecuarias y Ambientales",
    "AMBIENTAL": "Ciencias Agropecuarias y Ambientales",

    # --- 9. Otras Áreas (Menor Prioridad) ---
    "MILITARES": "Otras Áreas",
    "TEOLOGIA": "Otras Áreas",
    "NAVALES": "Otras Áreas",
    "NAUTICAS": "Otras Áreas",
    "DEPORTE": "Otras Áreas",
    "RECREACION": "Otras Áreas",
}

def asignar_area(programa):
    """
    Busca palabras clave del diccionario MAPEO_AREAS en el nombre
    del programa (ya normalizado) y devuelve el área correspondiente.
    """
    programa_norm = normalizar_texto(programa)

    for palabra_clave, area in MAPEO_AREAS.items():
        # se usa' in ' para buscar la palabra clave en cualquier parte del nombre
        if palabra_clave in programa_norm:
            return area

    # Si no encuentra ninguna coincidencia
    return "Área No Identificada"

z = pd.DataFrame(z)

# Aplica la función 'asignar_area' a cada fila de la columna 'E_PRGM_ACADEMICO'
# y guarda el resultado en la nueva columna 'F_AREA_CON_PRGM'
z['F_AREA_CON_PRGM'] = z['E_PRGM_ACADEMICO'].apply(asignar_area)


- La columna `RENDIMIENTO_GLOBAL` se generará utilizando la mayor parte de las demás columnas se utilizarán las siguientes y su correspondiente conversión:

  - `E_VALORMATRICULAUNIVERSIDAD`, que necesitaremos convertir a una **codificación continua**
  - `E_HORASSEMANATRABAJA`, que necesitaremos convertir a una **codificación continua**
  - `PERIODO_ACADEMICO`, que necesitaremos convertir a una **codificación onehot**
  - `E_PRGM_DEPARTAMENTO`, que necesitaremos convertir a una **codificación onehot**
  - `F_TIENEINTERNET`, que necesitaremos convertir a una **codificación onehot**
  - `F_EDUCACIONPADRE`, que necesitaremos convertir a una **codificación onehot**
  - `F_TIENELAVADORA`, que necesitaremos convertir a una **codificación onehot**
  - `F_TIENEAUTOMOVIL`, que necesitaremos convertir a una **codificación onehot**
  - `E_PAGOMATRICULAPROPIO`, que necesitaremos convertir a una **codificación onehot**
  - `F_TIENECOMPUTADOR`, que necesitaremos convertir a una **codificación onehot**
  - `F_EDUCACIONMADRE`, que necesitaremos convertir a una **codificación onehot**
  - `F_AREA_CON_PRGM`, que necesitaremos convertir a una **codificación onehot**

Se eliminarán del dataframe las siguientes columnas porque no aportan mayor información al entrenamiento del modelo:

 - `F_ESTRATOVIVIENDA`, que necesitaremos convertir a una **codificación onehot**
 - `F_TIENEINTERNET.1`
- `E_PRIVADO_LIBERTAD`, que necesitaremos convertir a una **codificación onehot**

## Limpieza Inicial



Sustitución de los valores `nan` por `'no info'` para marcarlos

In [12]:
# Sustitución de los valores `nan` con un símbolo preestablecido para marcarlos
z.E_PRGM_DEPARTAMENTO.values[z.E_PRGM_DEPARTAMENTO.isna()] = 'no info'
z.E_VALORMATRICULAUNIVERSIDAD.values[z.E_VALORMATRICULAUNIVERSIDAD.isna()] = 'no info'
z.E_HORASSEMANATRABAJA.values[z.E_HORASSEMANATRABAJA.isna()] = 'no info'
z.F_ESTRATOVIVIENDA.values[z.F_ESTRATOVIVIENDA.isna()] = 'no info'
z.F_TIENEINTERNET.values[z.F_TIENEINTERNET.isna()] = 'no info'
z.F_EDUCACIONPADRE.values[z.F_EDUCACIONPADRE.isna()] = 'no info'
z.F_TIENELAVADORA.values[z.F_TIENELAVADORA.isna()] = 'no info'
z.F_TIENEAUTOMOVIL.values[z.F_TIENEAUTOMOVIL.isna()] = 'no info'
z.E_PRIVADO_LIBERTAD.values[z.E_PRIVADO_LIBERTAD.isna()] = 'no info'
z.E_PAGOMATRICULAPROPIO.values[z.E_PAGOMATRICULAPROPIO.isna()] = 'no info'
z.F_TIENECOMPUTADOR.values[z.F_TIENECOMPUTADOR.isna()] = 'no info'
z.F_EDUCACIONMADRE.values[z.F_EDUCACIONMADRE.isna()] = 'no info'
z.F_AREA_CON_PRGM.values[z.F_AREA_CON_PRGM.isna()] = 'no info'

# Verificación de que ya no hay ningún valor NaN
sum(z.E_VALORMATRICULAUNIVERSIDAD.isna()), sum(z.E_HORASSEMANATRABAJA.isna()), sum(z.F_ESTRATOVIVIENDA.isna()), sum(z.F_TIENEINTERNET.isna()), sum(z.F_EDUCACIONPADRE.isna()), sum(z.F_TIENELAVADORA.isna()), sum(z.F_TIENEAUTOMOVIL.isna()), sum(z.E_PRIVADO_LIBERTAD.isna()), sum(z.E_PAGOMATRICULAPROPIO.isna()), sum(z.F_TIENECOMPUTADOR.isna()), sum(z.F_EDUCACIONMADRE.isna()), sum(z.F_AREA_CON_PRGM.isna())

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

### Conversión a valor continuo de `E_VALORMATRICULAUNIVERSIDAD`

Conversión `E_VALORMATRICULAUNIVERSIDAD` a un valor **continuo**, ya que el orden tiene sentido y se quiere que los modelos puedan capturarlo.

In [13]:
# Diccionario de phyton que mapea las categorías de texto a un número flotante que represente el punto medio del rango
cmap = {
 'No pagó matrícula': 0,
 'Menos de 500 mil': .25,
 'Entre 500 mil y menos de 1 millón': .75,
 'Entre 1 millón y menos de 2.5 millones': 1.75,
 'Entre 2.5 millones y menos de 4 millones': 3.25,
 'Entre 4 millones y menos de 5.5 millones': 4.75,
 'Entre 5.5 millones y menos de 7 millones': 6.25,
 'Más de 7 millones': 7.75,
 'no info': -1}

# Reemplaza la columna original con el array de números numpy, que se creo luego de reemplazar cada texto por el número específico
z.E_VALORMATRICULAUNIVERSIDAD = np.r_[[cmap[i] for i in z.E_VALORMATRICULAUNIVERSIDAD]]

# Realizar conteo de cuantas veces aparece cada valor numérico
z.E_VALORMATRICULAUNIVERSIDAD.value_counts()

,count
E_VALORMATRICULAUNIVERSIDAD,
1.75,204048
3.25,127430
0.25,80263
0.75,78704
4.75,69736
7.75,68014
6.25,38490
0.00,19528
-1.00,4579


### Conversión a valor continuo de `E_HORASSEMANATRABAJA`

Conversión `E_HORASSEMANATRABAJA` a un valor **continuo**, ya que el orden tiene sentido y se quiere que los modelos puedan capturarlo.

In [14]:
# Diccionario de phyton que mapea las categorías de texto a un número flotante que represente el punto medio del rango
cmap = {
 '0': 0,
 'Menos de 10 horas': 5,
 'Entre 11 y 20 horas': 15.5,
 'Entre 21 y 30 horas': 25.5,
 'Más de 30 horas': 35,
 'no info': -1}

# Reemplaza la columna original con el array de números numpy, que se creo luego de reemplazar cada texto por el número específico
z.E_HORASSEMANATRABAJA = np.r_[[cmap[i] for i in z.E_HORASSEMANATRABAJA]]

# Realizar conteo de cuantas veces aparece cada valor numérico
z.E_HORASSEMANATRABAJA.value_counts()

,count
E_HORASSEMANATRABAJA,
35.0,249347
0.0,116549
15.5,115856
25.5,92692
5.0,87190
-1.0,29158


### Conversión a valor continuo de `F_EDUCACIONMADRE`

Conversión `F_EDUCACIONMADRE` a un valor **continuo**, ya que el orden tiene sentido y se quiere que los modelos puedan capturarlo.

In [15]:
# Diccionario de phyton que mapea las categorías de texto a un número entero
cmap = {
 'Ninguno': 0,
 'Primaria incompleta': 1,
 'Primaria completa': 2,
 'Secundaria (Bachillerato) incompleta': 3,
 'Secundaria (Bachillerato) completa': 4,
 'Técnica o tecnológica incompleta': 5,
 'Técnica o tecnológica completa': 6,
 'Educación profesional incompleta': 7,
 'Educación profesional completa': 8,
 'Postgrado': 9,
 'no info': -1,
 'No sabe':-1,
 'No Aplica':-1 }

# Reemplaza la columna original con el array de números numpy, que se creó luego de reemplazar cada texto por el número específico
z.F_EDUCACIONMADRE = np.r_[[cmap[i] for i in z.F_EDUCACIONMADRE]]

# Realizar conteo de cuantas veces aparece cada valor numérico
z.F_EDUCACIONMADRE.value_counts()


,count
F_EDUCACIONMADRE,
4,141733
1,99414
6,89535
8,85319
3,81005
2,56121
9,46237
5,27531
-1,26948


### Conversión a valor continuo de `F_EDUCACIONPADRE`

Conversión `F_EDUCACIONPADRE` a un valor **continuo**, ya que el orden tiene sentido y se quiere que los modelos puedan capturarlo.

In [16]:
# Diccionario de phyton que mapea las categorías de texto a un número entero
cmap = {
 'Ninguno': 0,
 'Primaria incompleta': 1,
 'Primaria completa': 2,
 'Secundaria (Bachillerato) incompleta': 3,
 'Secundaria (Bachillerato) completa': 4,
 'Técnica o tecnológica incompleta': 5,
 'Técnica o tecnológica completa': 6,
 'Educación profesional incompleta': 7,
 'Educación profesional completa': 8,
 'Postgrado': 9,
 'no info': -1,
 'No sabe':-1,
 'No Aplica':-1 }

# Reemplaza la columna original con el array de números numpy, que se creó luego de reemplazar cada texto por el número específico
z.F_EDUCACIONPADRE = np.r_[[cmap[i] for i in z.F_EDUCACIONPADRE]]

# Realizar conteo de cuantas veces aparece cada valor numérico
z.F_EDUCACIONPADRE.value_counts()


,count
F_EDUCACIONPADRE,
4,128281
1,125660
8,83104
3,71647
6,62990
2,55955
-1,47354
9,44163
7,27081


### `PERIODO_ACADEMICO` se parte en año y período

In [17]:
import pandas as pd

# Asegurar que la columna sea de tipo 'string'
# Esto permite cortarla de forma segura
z.PERIODO_ACADEMICO = z.PERIODO_ACADEMICO.astype(str)

# Extraer el Año (los primeros 4 caracteres)
z['anio'] = z.PERIODO_ACADEMICO.str[0:4]

# Extraer el SubPeriodo (del 4to caracter en adelante)
z['periodo'] = z.PERIODO_ACADEMICO.str[4:]

# Ahora se tiene dos nuevas columnas, pero AMBAS son strings.

# Se convierte el año a entero (int)
z['anio'] = z['anio'].astype(int)

# Se convierte el período a 'category' para que el modelo lo trate como tal
z['periodo']  = z['periodo'].astype('category')

print("\n--- DATOS TRANSFORMADOS ---")
print(z)

print("\n--- NUEVOS TIPOS DE DATOS ---")
print(z.info())


--- DATOS TRANSFORMADOS ---
            ID PERIODO_ACADEMICO                     E_PRGM_ACADEMICO  \
0       904256             20212                           ENFERMERIA   
1       645256             20212                              DERECHO   
2       308367             20203                MERCADEO Y PUBLICIDAD   
3       470353             20195           ADMINISTRACION DE EMPRESAS   
4       989032             20212                           PSICOLOGIA   
...        ...               ...                                  ...   
690787   25096             20195                             BIOLOGIA   
690788  754213             20212                           PSICOLOGIA   
690789  504185             20183  ADMINISTRACIÓN EN SALUD OCUPACIONAL   
690790  986620             20195                           PSICOLOGIA   
690791  933374             20195                           PSICOLOGIA   

       E_PRGM_DEPARTAMENTO  E_VALORMATRICULAUNIVERSIDAD  E_HORASSEMANATRABAJA  \
0            

### Crear el mapeo onehot para la nueva columna `periodo`

In [18]:
x = z.periodo.values
periodo_vals = sorted(np.unique(x))
periodo_onehot_vals = {val: np.eye(len(periodo_vals))[i] for i,val in enumerate(periodo_vals)}

periodo_onehot_enc = np.r_[[periodo_onehot_vals[i] for i in z.periodo]]

periodo_df = pd.DataFrame(periodo_onehot_enc, columns=[f"periodo__{v}" for v in periodo_onehot_vals])

## Mapeo onehot para la columna `PERIODO_ACADEMICO`

In [19]:
x = z.PERIODO_ACADEMICO.values
PERIODO_ACADEMICO_vals = sorted(np.unique(x))
PERIODO_ACADEMICO_onehot_vals = {val: np.eye(len(PERIODO_ACADEMICO_vals))[i] for i,val in enumerate(PERIODO_ACADEMICO_vals)}
PERIODO_ACADEMICO_onehot_enc = np.r_[[PERIODO_ACADEMICO_onehot_vals[i] for i in z.PERIODO_ACADEMICO]]
PERIODO_ACADEMICO_df = pd.DataFrame(PERIODO_ACADEMICO_onehot_enc, columns=[f"PERIODO_ACADEMICO__{v}" for v in PERIODO_ACADEMICO_onehot_vals])

In [20]:
PERIODO_ACADEMICO_df

,PERIODO_ACADEMICO__20183,PERIODO_ACADEMICO__20184,PERIODO_ACADEMICO__20194,PERIODO_ACADEMICO__20195,PERIODO_ACADEMICO__20196,PERIODO_ACADEMICO__20202,PERIODO_ACADEMICO__20203,PERIODO_ACADEMICO__20212,PERIODO_ACADEMICO__20213
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
690787,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
690788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
690789,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
690790,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Crear el mapeo onehot de `E_PRGM_DEPARTAMENTO`

In [21]:
x = z.E_PRGM_DEPARTAMENTO.values
E_PRGM_DEPARTAMENTO_vals = sorted(np.unique(x))
E_PRGM_DEPARTAMENTO_onehot_vals = {val: np.eye(len(E_PRGM_DEPARTAMENTO_vals))[i] for i,val in enumerate(E_PRGM_DEPARTAMENTO_vals)}
E_PRGM_DEPARTAMENTO_onehot_vals

{'AMAZONAS': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'ANTIOQUIA': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'ARAUCA': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'ATLANTICO': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'BOGOTÁ': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'BOLIVAR': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'BOYACA': array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [22]:
E_PRGM_DEPARTAMENTO_onehot_enc = np.r_[[E_PRGM_DEPARTAMENTO_onehot_vals[i] for i in z.E_PRGM_DEPARTAMENTO]]
E_PRGM_DEPARTAMENTO_onehot_enc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [23]:
E_PRGM_DEPARTAMENTO_df = pd.DataFrame(E_PRGM_DEPARTAMENTO_onehot_enc, columns=[f"E_PRGM_DEPARTAMENTO__{v}" for v in E_PRGM_DEPARTAMENTO_onehot_vals])
E_PRGM_DEPARTAMENTO_df

,E_PRGM_DEPARTAMENTO__AMAZONAS,E_PRGM_DEPARTAMENTO__ANTIOQUIA,E_PRGM_DEPARTAMENTO__ARAUCA,E_PRGM_DEPARTAMENTO__ATLANTICO,E_PRGM_DEPARTAMENTO__BOGOTÁ,E_PRGM_DEPARTAMENTO__BOLIVAR,E_PRGM_DEPARTAMENTO__BOYACA,E_PRGM_DEPARTAMENTO__CALDAS,E_PRGM_DEPARTAMENTO__CAQUETA,E_PRGM_DEPARTAMENTO__CASANARE,...,E_PRGM_DEPARTAMENTO__NORTE SANTANDER,E_PRGM_DEPARTAMENTO__PUTUMAYO,E_PRGM_DEPARTAMENTO__QUINDIO,E_PRGM_DEPARTAMENTO__RISARALDA,E_PRGM_DEPARTAMENTO__SAN ANDRES,E_PRGM_DEPARTAMENTO__SANTANDER,E_PRGM_DEPARTAMENTO__SUCRE,E_PRGM_DEPARTAMENTO__TOLIMA,E_PRGM_DEPARTAMENTO__VALLE,E_PRGM_DEPARTAMENTO__VAUPES
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
690788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
690789,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
690790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Crear el mapeo onehot de `F_TIENEINTERNET`



In [24]:
x = z.F_TIENEINTERNET.values
F_TIENEINTERNET_vals = sorted(np.unique(x))
F_TIENEINTERNET_onehot_vals = {val: np.eye(len(F_TIENEINTERNET_vals))[i] for i,val in enumerate(F_TIENEINTERNET_vals)}
F_TIENEINTERNET_onehot_vals

{'No': array([1., 0., 0.]),
 'Si': array([0., 1., 0.]),
 'no info': array([0., 0., 1.])}

In [25]:
F_TIENEINTERNET_onehot_enc = np.r_[[F_TIENEINTERNET_onehot_vals[i] for i in z.F_TIENEINTERNET]]
F_TIENEINTERNET_onehot_enc

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [26]:
F_TIENEINTERNET_df = pd.DataFrame(F_TIENEINTERNET_onehot_enc, columns=[f"F_TIENEINTERNET__{v}" for v in F_TIENEINTERNET_onehot_vals])
F_TIENEINTERNET_df

,F_TIENEINTERNET__No,F_TIENEINTERNET__Si,F_TIENEINTERNET__no info
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
690787,0.0,1.0,0.0
690788,0.0,1.0,0.0
690789,0.0,1.0,0.0
690790,1.0,0.0,0.0


### Crear el mapeo onehot de `E_PAGOMATRICULAPROPIO`

In [27]:
x = z.E_PAGOMATRICULAPROPIO.values
E_PAGOMATRICULAPROPIO_vals = sorted(np.unique(x))
E_PAGOMATRICULAPROPIO_onehot_vals = {val: np.eye(len(E_PAGOMATRICULAPROPIO_vals))[i] for i,val in enumerate(E_PAGOMATRICULAPROPIO_vals)}
E_PAGOMATRICULAPROPIO_onehot_vals

{'No': array([1., 0., 0.]),
 'Si': array([0., 1., 0.]),
 'no info': array([0., 0., 1.])}

In [28]:
E_PAGOMATRICULAPROPIO_onehot_enc = np.r_[[E_PAGOMATRICULAPROPIO_onehot_vals[i] for i in z.E_PAGOMATRICULAPROPIO]]
E_PAGOMATRICULAPROPIO_onehot_enc

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [29]:
E_PAGOMATRICULAPROPIO_df = pd.DataFrame(E_PAGOMATRICULAPROPIO_onehot_enc, columns=[f"E_PAGOMATRICULAPROPIO__{v}" for v in E_PAGOMATRICULAPROPIO_onehot_vals])
E_PAGOMATRICULAPROPIO_df

,E_PAGOMATRICULAPROPIO__No,E_PAGOMATRICULAPROPIO__Si,E_PAGOMATRICULAPROPIO__no info
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
690787,0.0,1.0,0.0
690788,1.0,0.0,0.0
690789,0.0,1.0,0.0
690790,0.0,1.0,0.0


### Crear el mapeo onehot de `F_TIENECOMPUTADOR`

In [30]:
x = z.F_TIENECOMPUTADOR.values
F_TIENECOMPUTADOR_vals = sorted(np.unique(x))
F_TIENECOMPUTADOR_onehot_vals = {val: np.eye(len(F_TIENECOMPUTADOR_vals))[i] for i,val in enumerate(F_TIENECOMPUTADOR_vals)}
F_TIENECOMPUTADOR_onehot_vals

{'No': array([1., 0., 0.]),
 'Si': array([0., 1., 0.]),
 'no info': array([0., 0., 1.])}

In [31]:
F_TIENECOMPUTADOR_onehot_enc = np.r_[[F_TIENECOMPUTADOR_onehot_vals[i] for i in z.F_TIENECOMPUTADOR]]
F_TIENECOMPUTADOR_onehot_enc

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [32]:
F_TIENECOMPUTADOR_df = pd.DataFrame(F_TIENECOMPUTADOR_onehot_enc, columns=[f"F_TIENECOMPUTADOR__{v}" for v in F_TIENECOMPUTADOR_onehot_vals])
F_TIENECOMPUTADOR_df

,F_TIENECOMPUTADOR__No,F_TIENECOMPUTADOR__Si,F_TIENECOMPUTADOR__no info
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
690787,0.0,1.0,0.0
690788,0.0,1.0,0.0
690789,0.0,1.0,0.0
690790,0.0,1.0,0.0


### Crear el mapeo onehot de `F_AREA_CON_PRGM`

In [33]:
x = z.F_AREA_CON_PRGM.values
F_AREA_CON_PRGM_vals = sorted(np.unique(x))
F_AREA_CON_PRGM_onehot_vals = {val: np.eye(len(F_AREA_CON_PRGM_vals))[i] for i,val in enumerate(F_AREA_CON_PRGM_vals)}
F_AREA_CON_PRGM_onehot_vals

{'Artes, Diseño y Comunicación': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Ciencias Agropecuarias y Ambientales': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Ciencias Básicas y Naturales': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 'Ciencias Económicas y Administrativas': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 'Ciencias Sociales y Humanas': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 'Ciencias de la Salud': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 'Educación (Licenciaturas)': array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 'Ingeniería y Tecnología': array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]),
 'Otras Áreas': array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 'Área No Identificada': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])}

In [34]:
F_AREA_CON_PRGM_onehot_enc = np.r_[[F_AREA_CON_PRGM_onehot_vals[i] for i in z.F_AREA_CON_PRGM]]
F_AREA_CON_PRGM_onehot_enc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
F_AREA_CON_PRGM_df = pd.DataFrame(F_AREA_CON_PRGM_onehot_enc, columns=[f"F_AREA_CON_PRGM__{v}" for v in F_AREA_CON_PRGM_onehot_vals])
F_AREA_CON_PRGM_df

,"F_AREA_CON_PRGM__Artes, Diseño y Comunicación",F_AREA_CON_PRGM__Ciencias Agropecuarias y Ambientales,F_AREA_CON_PRGM__Ciencias Básicas y Naturales,F_AREA_CON_PRGM__Ciencias Económicas y Administrativas,F_AREA_CON_PRGM__Ciencias Sociales y Humanas,F_AREA_CON_PRGM__Ciencias de la Salud,F_AREA_CON_PRGM__Educación (Licenciaturas),F_AREA_CON_PRGM__Ingeniería y Tecnología,F_AREA_CON_PRGM__Otras Áreas,F_AREA_CON_PRGM__Área No Identificada
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
690787,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
690788,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
690789,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
690790,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### Crear el mapeo onehot de `F_TIENELAVADORA`

In [36]:
x = z.F_TIENELAVADORA.values
F_TIENELAVADORA_vals = sorted(np.unique(x))
F_TIENELAVADORA_onehot_vals = {val: np.eye(len(F_TIENELAVADORA_vals))[i] for i,val in enumerate(F_TIENELAVADORA_vals)}
F_TIENELAVADORA_onehot_vals

{'No': array([1., 0., 0.]),
 'Si': array([0., 1., 0.]),
 'no info': array([0., 0., 1.])}

In [37]:
F_TIENELAVADORA_onehot_enc = np.r_[[F_TIENELAVADORA_onehot_vals[i] for i in z.F_TIENELAVADORA]]
F_TIENELAVADORA_onehot_enc

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [38]:
F_TIENELAVADORA_df = pd.DataFrame(F_TIENELAVADORA_onehot_enc, columns=[f"F_TIENELAVADORA__{v}" for v in F_TIENELAVADORA_onehot_vals])
F_TIENELAVADORA_df

,F_TIENELAVADORA__No,F_TIENELAVADORA__Si,F_TIENELAVADORA__no info
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
690787,0.0,1.0,0.0
690788,0.0,1.0,0.0
690789,0.0,1.0,0.0
690790,1.0,0.0,0.0


### Crear el mapeo onehot de `F_TIENEAUTOMOVIL`

In [39]:
x = z.F_TIENEAUTOMOVIL.values
F_TIENEAUTOMOVIL_vals = sorted(np.unique(x))
F_TIENEAUTOMOVIL_onehot_vals = {val: np.eye(len(F_TIENEAUTOMOVIL_vals))[i] for i,val in enumerate(F_TIENEAUTOMOVIL_vals)}
F_TIENEAUTOMOVIL_onehot_vals

{'No': array([1., 0., 0.]),
 'Si': array([0., 1., 0.]),
 'no info': array([0., 0., 1.])}

In [40]:
F_TIENEAUTOMOVIL_onehot_enc = np.r_[[F_TIENEAUTOMOVIL_onehot_vals[i] for i in z.F_TIENEAUTOMOVIL]]
F_TIENEAUTOMOVIL_onehot_enc

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [41]:
F_TIENEAUTOMOVIL_df = pd.DataFrame(F_TIENEAUTOMOVIL_onehot_enc, columns=[f"F_TIENEAUTOMOVIL__{v}" for v in F_TIENEAUTOMOVIL_onehot_vals])
F_TIENEAUTOMOVIL_df

,F_TIENEAUTOMOVIL__No,F_TIENEAUTOMOVIL__Si,F_TIENEAUTOMOVIL__no info
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0
...,...,...,...
690787,1.0,0.0,0.0
690788,1.0,0.0,0.0
690789,1.0,0.0,0.0
690790,1.0,0.0,0.0


In [42]:
print(z.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690792 entries, 0 to 690791
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   ID                           690792 non-null  int64   
 1   PERIODO_ACADEMICO            690792 non-null  object  
 2   E_PRGM_ACADEMICO             690792 non-null  object  
 3   E_PRGM_DEPARTAMENTO          690792 non-null  object  
 4   E_VALORMATRICULAUNIVERSIDAD  690792 non-null  float64 
 5   E_HORASSEMANATRABAJA         690792 non-null  float64 
 6   F_ESTRATOVIVIENDA            690792 non-null  object  
 7   F_TIENEINTERNET              690792 non-null  object  
 8   F_EDUCACIONPADRE             690792 non-null  int64   
 9   F_TIENELAVADORA              690792 non-null  object  
 10  F_TIENEAUTOMOVIL             690792 non-null  object  
 11  E_PRIVADO_LIBERTAD           690792 non-null  object  
 12  E_PAGOMATRICULAPROPIO        690792 non-null

### Eliminación de dataframes originales y concatenación de los nuevos DataFrames 'One-hot'

Se junta todo en un solo dataframe, eliminando las columnas originales que se limpiaron y que se convirtieron en DataFrames 'One-hot'.

Se tienen columnas binarias a las que se le asigna un valor de 1 a solo una de ellas por cada fila, de acuerdo al valor asociado a cada uno de los estudiantes con el DataFrame original y sus correspondientes columnas.

In [43]:
# Lista de todas las columnas originales que se van a eliminar
columnas_originales = ['ID', 'E_PRGM_ACADEMICO','E_PRGM_DEPARTAMENTO', 'F_TIENEINTERNET','F_TIENELAVADORA','F_TIENEAUTOMOVIL',
'E_PRIVADO_LIBERTAD','E_PAGOMATRICULAPROPIO','F_TIENECOMPUTADOR','F_TIENEINTERNET.1', 'F_AREA_CON_PRGM', 'INDICADOR_3', 'INDICADOR_4', 'F_ESTRATOVIVIENDA']

# Lista de TODOS los DataFrames que se van a unir
dataframes_para_unir = [
    z,
     F_TIENEINTERNET_df, F_TIENECOMPUTADOR_df, F_AREA_CON_PRGM_df, F_TIENELAVADORA_df, F_TIENEAUTOMOVIL_df, E_PAGOMATRICULAPROPIO_df
    ]

# Concatena todo: z + todos los nuevos DataFrames
z_final = pd.concat(dataframes_para_unir, axis=1)

# Elimina todas las columnas originales
z_final = z_final.drop(columns=columnas_originales)

# Revisar el resultado de la concatenación
print(z_final.shape)
z_final

(690792, 35)


,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_EDUCACIONPADRE,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,anio,periodo,...,F_AREA_CON_PRGM__Área No Identificada,F_TIENELAVADORA__No,F_TIENELAVADORA__Si,F_TIENELAVADORA__no info,F_TIENEAUTOMOVIL__No,F_TIENEAUTOMOVIL__Si,F_TIENEAUTOMOVIL__no info,E_PAGOMATRICULAPROPIO__No,E_PAGOMATRICULAPROPIO__Si,E_PAGOMATRICULAPROPIO__no info
0,20212,6.25,5.0,5,9,medio-alto,0.322,0.208,2021,2,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,20212,3.25,0.0,6,5,bajo,0.311,0.215,2021,2,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,20203,3.25,35.0,4,4,bajo,0.297,0.214,2020,3,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,20195,4.75,0.0,-1,4,alto,0.485,0.172,2019,5,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,20212,3.25,25.5,2,2,medio-bajo,0.316,0.232,2021,2,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690787,20195,0.75,15.5,4,3,medio-alto,0.237,0.271,2019,5,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
690788,20212,3.25,35.0,1,3,bajo,0.314,0.240,2021,2,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
690789,20183,1.75,5.0,4,3,medio-bajo,0.286,0.240,2018,3,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
690790,20195,3.25,5.0,2,2,bajo,0.132,0.426,2019,5,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [44]:
z_final = z_final.drop(columns=['periodo'])

In [45]:
print(z_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690792 entries, 0 to 690791
Data columns (total 34 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   PERIODO_ACADEMICO                                       690792 non-null  object 
 1   E_VALORMATRICULAUNIVERSIDAD                             690792 non-null  float64
 2   E_HORASSEMANATRABAJA                                    690792 non-null  float64
 3   F_EDUCACIONPADRE                                        690792 non-null  int64  
 4   F_EDUCACIONMADRE                                        690792 non-null  int64  
 5   RENDIMIENTO_GLOBAL                                      690792 non-null  object 
 6   INDICADOR_1                                             690792 non-null  float64
 7   INDICADOR_2                                             690792 non-null  float64
 8   anio                    

## Convertir la columna objetivo (predicción) en valores discretos.

In [46]:
y_col = 'RENDIMIENTO_GLOBAL'
rmap = {'alto': 3, 'bajo':0, 'medio-bajo':1, 'medio-alto':2}
z_final[y_col] = [rmap[i] for i in z_final[y_col]]
z_final.head()

,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_EDUCACIONPADRE,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,anio,F_TIENEINTERNET__No,...,F_AREA_CON_PRGM__Área No Identificada,F_TIENELAVADORA__No,F_TIENELAVADORA__Si,F_TIENELAVADORA__no info,F_TIENEAUTOMOVIL__No,F_TIENEAUTOMOVIL__Si,F_TIENEAUTOMOVIL__no info,E_PAGOMATRICULAPROPIO__No,E_PAGOMATRICULAPROPIO__Si,E_PAGOMATRICULAPROPIO__no info
0,20212,6.25,5.0,5,9,2,0.322,0.208,2021,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,20212,3.25,0.0,6,5,0,0.311,0.215,2021,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,20203,3.25,35.0,4,4,0,0.297,0.214,2020,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,20195,4.75,0.0,-1,4,3,0.485,0.172,2019,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,20212,3.25,25.5,2,2,1,0.316,0.232,2021,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


## Se construye `X` e `y` para entrenar un modelo

Se ordenan las columnas para asegurarnos de que siempre tengan el mismo orden.

In [47]:
# Reordena alfabéticamente las columnas del DataFrame z_final
z_final = z_final[sorted(z_final.columns)]

# Crea array NumPy de características X, crea una lista de todas las columnas menos la variable objetivo 'RENDIMIENTO_GLOBAL'
X = z_final[[c for c in z_final.columns if c!=y_col]].values

# Selecciona la columna objetivo y la convierte en un array Numpy
y = z_final[y_col].values

# Verificar las dimensiones de los arrays NumPy
X.shape, y.shape

((690792, 33), (690792,))

## Dividir en conjunto de entrenamiento y probar

In [48]:
# Importar la herramienta train_test_split de la librería Scikit-learn para dividir datos
from sklearn.model_selection import train_test_split

In [49]:
# Para asignar el 80% de los datos al conjunto de entrenamiento y el 20% restante se usará para calificar al modelo
Xtr, Xts, ytr, yts = train_test_split(X,y, train_size=0.8,
                                      random_state=42, # Para reproducibilidad
                                      stratify=y)       # Asegura que 'y' esté balanceada)

# Imprime las dimensiones de los cuatro nuevos conjuntos de datos
Xtr.shape, Xts.shape, ytr.shape, yts.shape

((552633, 33), (138159, 33), (552633,), (138159,))

# **Crear un modelo con ramdom forest**

In [50]:
# Importar RandomForestClassifier desde la librería sklearn
from sklearn.ensemble import RandomForestClassifier

# Crea el modelo con 200 árboles de decisión
random_state=42 #permite que el resultado sea reproducible
n_jobs=-1 #usa todos los núcleos disponibles
rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=12,       # Árboles más simples y rápidos
    min_samples_leaf=10,# Evita cálculos en ramas muy finas
    max_samples=0.5,    # Cada árbol se entrena solo con el 50% de los datos al azar
    n_jobs=-1,          # Usa todos los núcleos
    random_state=42,
    class_weight='balanced'
)

# Se entrena al modelo
rf.fit(Xtr, ytr)

RandomForestClassifier(class_weight='balanced', max_depth=12, max_samples=0.5,
                       min_samples_leaf=10, n_estimators=50, n_jobs=-1,
                       random_state=42)

Obtener predicciones

In [51]:
preds_tr = rf.predict(Xtr)
preds_ts = rf.predict(Xts)

In [54]:
from sklearn.metrics import accuracy_score, classification_report

# El modelo intenta adivinar el rendimiento basándose en lo que aprendió
y_pred = rf.predict(Xts)

# Calcular la precisión comparando la predicción (y_pred) con la realidad (yts)
precision = accuracy_score(yts, y_pred)

print(f"Precisión global del modelo (Accuracy): {precision * 100:.2f}%")

# Reporte por clase
print("\n--- Desglose por tipo de rendimiento ---")
print(classification_report(yts, y_pred))

✅ Precisión global del modelo (Accuracy): 40.21%

--- Desglose por tipo de rendimiento ---
              precision    recall  f1-score   support

           0       0.41      0.55      0.47     34511
           1       0.33      0.25      0.28     34369
           2       0.30      0.20      0.24     34240
           3       0.50      0.61      0.55     35039

    accuracy                           0.40    138159
   macro avg       0.38      0.40      0.38    138159
weighted avg       0.38      0.40      0.38    138159



## Se mantiene el primer modelo; ahora se aplica el mismo procedimiento a `test.csv`

Observación:
- No existe la columna `RENDIMIENTO_GLOBAL`
- Se debe conservar los ID para poder crear correctamente el archivo de envío

In [98]:
zt = pd.read_csv("test.csv")
zt

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,496981,20195,ADMINISTRACION DE EMPRESAS,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 1,Si,Primaria incompleta,Si,Si,N,Si,Si,Si,Primaria incompleta,0.168,0.410,0.235,0.300
296782,209415,20183,DERECHO,META,Entre 1 millón y menos de 2.5 millones,0,Estrato 4,Si,Educación profesional completa,Si,No,N,No,Si,Si,Educación profesional completa,0.471,0.184,0.264,0.193
296783,239074,20212,DERECHO,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Educación profesional completa,0.292,0.249,0.276,0.256
296784,963852,20195,INGENIERIA AERONAUTICA,ANTIOQUIA,Entre 5.5 millones y menos de 7 millones,Entre 11 y 20 horas,Estrato 3,Si,Educación profesional completa,Si,No,N,No,Si,Si,Educación profesional completa,0.305,0.219,0.310,0.260


# **PREPROCESADO DE DATOS**

## Eliminar filas con muchos valores nulos

In [99]:
import numpy as np

cols = [
    "E_VALORMATRICULAUNIVERSIDAD","E_HORASSEMANATRABAJA","F_ESTRATOVIVIENDA",
    "F_TIENEINTERNET","F_EDUCACIONPADRE","F_TIENELAVADORA","F_TIENEAUTOMOVIL","E_PAGOMATRICULAPROPIO",
    "F_TIENECOMPUTADOR","F_EDUCACIONMADRE"
]

# Reemplazar indicadores comunes de "vacío" por NaN
zt[cols] = zt[cols].replace({'': np.nan, 'NA': np.nan, 'N/A': np.nan, '-': np.nan})

# Limpiar celdas que solo tengan espacios
zt[cols] = zt[cols].replace(r'^\s*$', np.nan, regex=True)

# Define columnas obligatorias y el umbral, se puso como obligatoria la columna E_VALORMATRICULAUNIVERSIDAD por la importancia que
# ha tenido en la mayor parte de los indicadores
col_obligatoria = "E_VALORMATRICULAUNIVERSIDAD"

# Obtenemos la lista de las otras 9 columnas "opcionales"
cols_opcionales = [col for col in cols if col != col_obligatoria]

# Se define cuántos valores NO NULOS deben existir en las 9 columnas opcionales
# para salvar una fila si la columna obligatoria (E_VALORMATRICULAUNIVERSIDAD) Es nula.

# Por ejemplo: Si se pone 5, significa: "Si la matrícula es nula,
#           conserva la fila si al menos 5 de las otras 9 columnas tiene un valor no nulo".
min_non_na_opcionales = 5

# --- Construcción de las condiciones de filtrado ---

# Condición 1: Conservar si la columna obligatoria NO es nula
condicion_1 = zt[col_obligatoria].notnull()

# Condición 2: Conservar si la columna obligatoria sí es nula,
# Pero las columnas opcionales cumplen el umbral mínimo de valores no nulos.
condicion_2 = (zt[col_obligatoria].isnull()) & \
              (zt[cols_opcionales].notnull().sum(axis=1) >= min_non_na_opcionales)

print("Dimensiones del dataframe antes de la depuración")
print(zt.shape)
# Conservamos las filas que cumplen la Condición 1 o la Condición 2
zt = zt[condicion_1 | condicion_2].reset_index(drop=True)

# Resultado
print("Dimensiones del dataframe depurado")
print(zt.shape)

Dimensiones del dataframe antes de la depuración
(296786, 20)
Dimensiones del dataframe depurado
(296035, 20)


## Creación de nueva columna en el Dataframe que agrupa en áreas de conocimiento los diferentes programas de la columna E_PRGM_ACADEMICO

In [100]:
import pandas as pd
import unicodedata
import re

# Definición de Funciones de Limpieza y Mapeo

def corregir_codificacion_consorcio(texto):
    """
    Corrige errores comunes de codificación (Mojibake) donde
    UTF-8 se malinterpretó como latin-1 o windows-1252.
    Ej: 'ADMINISTRACIÃ“N' -> 'ADMINISTRACIÓN'
    """
    if not isinstance(texto, str):
        return texto
    try:
        # Intenta decodificar de latin-1 y recodificar a UTF-8
        return texto.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        # Si falla, devuelve el texto original
        return texto

def normalizar_texto(texto):
    """
    Limpia un string:
    1. Corrige la codificación
    2. Convierte a mayúsculas.
    3. Elimina acentos.
    """
    if not isinstance(texto, str):
        return ""

    # Paso 1: Corregir codificación
    texto = corregir_codificacion_consorcio(texto)

    # Paso 2: Convertir a mayúsculas
    texto = texto.upper()

    # Paso 3: Eliminar acentos (Normalización NFD)
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )

    return texto.strip()

# --- Diccionario de Palabras Clave y Áreas (Priorizado) ---
MAPEO_AREAS = {
    # --- 1. Educación ---
    "LICENCIATURA": "Educación (Licenciaturas)",
    "PEDAGOGIA": "Educación (Licenciaturas)",
    "ETNOEDUCACION": "Educación (Licenciaturas)",

    # --- 2. Ciencias de la Salud ---
    "ENFERMERIA": "Ciencias de la Salud",
    "MEDICINA": "Ciencias de la Salud",
    "ODONTOLOGIA": "Ciencias de la Salud",
    "PSICOLOGIA": "Ciencias de la Salud",
    "SALUD": "Ciencias de la Salud",
    "BACTERIOLOGIA": "Ciencias de la Salud",
    "FISIOTERAPIA": "Ciencias de la Salud",
    "NUTRICION": "Ciencias de la Salud",
    "INSTRUMENTACION QUIRURGICA": "Ciencias de la Salud",
    "OPTOMETRIA": "Ciencias de la Salud",
    "FONOAUDIOLOGIA": "Ciencias de la Salud",
    "TERAPIA": "Ciencias de la Salud",
    "VETERINARIA": "Ciencias de la Salud",
    "ZOOTECNIA": "Ciencias de la Salud",
    "BIOINGENIERIA": "Ciencias de la Salud",
    "BIOMEDICA": "Ciencias de la Salud",
    "GERONTOLOGIA": "Ciencias de la Salud",

    # --- 3. Ingeniería y Tecnología ---
    "INGENIERIA": "Ingeniería y Tecnología",
    "ARQUITECTURA": "Ingeniería y Tecnología",
    "SISTEMAS": "Ingeniería y Tecnología",
    "SOFTWARE": "Ingeniería y Tecnología",
    "COMPUTACION": "Ingeniería y Tecnología",
    "TELECOMUNICACIONES": "Ingeniería y Tecnología",
    "MECATRONICA": "Ingeniería y Tecnología",
    "ELECTRONICA": "Ingeniería y Tecnología",
    "MECANICA": "Ingeniería y Tecnología",
    "CIVIL": "Ingeniería y Tecnología",
    "TELEMATICA": "Ingeniería y Tecnología",
    "MULTIMEDIA": "Ingeniería y Tecnología",
    "URBANISMO": "Ingeniería y Tecnología",
    "CONSTRUCCION": "Ingeniería y Tecnología",
    "AERONAUTICA": "Ingeniería y Tecnología",
    "NANOTECNOLOGIA": "Ingeniería y Tecnología",
    "AUTOMATIZACION": "Ingeniería y Tecnología",

    # --- 4. Ciencias Económicas y Administrativas ---
    "ADMINISTRACION": "Ciencias Económicas y Administrativas",
    "ECONOMIA": "Ciencias Económicas y Administrativas",
    "CONTADURIA": "Ciencias Económicas y Administrativas",
    "FINANZAS": "Ciencias Económicas y Administrativas",
    "FINANCIERA": "Ciencias Económicas y Administrativas",
    "MERCADEO": "Ciencias Económicas y Administrativas",
    "PUBLICIDAD": "Ciencias Económicas y Administrativas",
    "NEGOCIOS": "Ciencias Económicas y Administrativas",
    "COMERCIO": "Ciencias Económicas y Administrativas",
    "HOTELERIA": "Ciencias Económicas y Administrativas",
    "TURISMO": "Ciencias Económicas y Administrativas",
    "LOGISTICA": "Ciencias Económicas y Administrativas",
    "BANCARIA": "Ciencias Económicas y Administrativas",
    "INDUSTRIAL": "Ciencias Económicas y Administrativas",

    # --- 5. Ciencias Sociales y Humanas ---
    "DERECHO": "Ciencias Sociales y Humanas",
    "JURISPRUDENCIA": "Ciencias Sociales y Humanas",
    "CIENCIA POLITICA": "Ciencias Sociales y Humanas",
    "TRABAJO SOCIAL": "Ciencias Sociales y Humanas",
    "SOCIOLOGIA": "Ciencias Sociales y Humanas",
    "ANTROPOLOGIA": "Ciencias Sociales y Humanas",
    "HISTORIA": "Ciencias Sociales y Humanas",
    "FILOSOFIA": "Ciencias Sociales y Humanas",
    "RELACIONES INTERNACIONALES": "Ciencias Sociales y Humanas",
    "GEOGRAFIA": "Ciencias Sociales y Humanas",
    "HUMANIDADES": "Ciencias Sociales y Humanas",
    "SOCIALES": "Ciencias Sociales y Humanas",
    "POLITICA": "Ciencias Sociales y Humanas",
    "GOBIERNO": "Ciencias Sociales y Humanas",
    "CRIMINALISTICA": "Ciencias Sociales y Humanas",
    "ARCHIVISTICA": "Ciencias Sociales y Humanas",
    "BIBLIOTECOLOGIA": "Ciencias Sociales y Humanas",
    "LENGUAS": "Ciencias Sociales y Humanas",
    "FILOLOGIA": "Ciencias Sociales y Humanas",
    "LETRAS": "Ciencias Sociales y Humanas",
    "LITERATURA": "Ciencias Sociales y Humanas",
    "LINGUISTICA": "Ciencias Sociales y Humanas",

    # --- 6. Artes, Diseño y Comunicación ---
    "DISEÑO": "Artes, Diseño y Comunicación",
    "COMUNICACION": "Artes, Diseño y Comunicación",
    "PERIODISMO": "Artes, Diseño y Comunicación",
    "ARTE": "Artes, Diseño y Comunicación",
    "ARTES": "Artes, Diseño y Comunicación",
    "MUSICA": "Artes, Diseño y Comunicación",
    "CINE": "Artes, Diseño y Comunicación",
    "AUDIOVISUAL": "Artes, Diseño y Comunicación",
    "FOTOGRAFIA": "Artes, Diseño y Comunicación",
    "DANZA": "Artes, Diseño y Comunicación",
    "TEATRO": "Artes, Diseño y Comunicación",
    "GASTRONOMIA": "Artes, Diseño y Comunicación",
    "CULINARIA": "Artes, Diseño y Comunicación",
    "MODAS": "Artes, Diseño y Comunicación",

    # --- 7. Ciencias Básicas y Naturales ---
    "BIOLOGIA": "Ciencias Básicas y Naturales",
    "QUIMICA": "Ciencias Básicas y Naturales",
    "FISICA": "Ciencias Básicas y Naturales",
    "MATEMATICAS": "Ciencias Básicas y Naturales",
    "ESTADISTICA": "Ciencias Básicas y Naturales",
    "GEOLOGIA": "Ciencias Básicas y Naturales",
    "ECOLOGIA": "Ciencias Básicas y Naturales",
    "ASTRONOMIA": "Ciencias Básicas y Naturales",
    "MICROBIOLOGIA": "Ciencias Básicas y Naturales",
    "GEOCIENCIAS": "Ciencias Básicas y Naturales",
    "BIOTECNOLOGIA": "Ciencias Básicas y Naturales",
    "BIOQUIMICA": "Ciencias Básicas y Naturales",

    # --- 8. Ciencias Agropecuarias y Ambientales ---
    "AGRONOMIA": "Ciencias Agropecuarias y Ambientales",
    "AGROPECUARIA": "Ciencias Agropecuarias y Ambientales",
    "AGROINDUSTRIAL": "Ciencias Agropecuarias y Ambientales",
    "AGRONEGOCIOS": "Ciencias Agropecuarias y Ambientales",
    "PECUARIA": "Ciencias Agropecuarias y Ambientales",
    "ACUICULTURA": "Ciencias Agropecuarias y Ambientales",
    "FORESTAL": "Ciencias Agropecuarias y Ambientales",
    "AGRICOLA": "Ciencias Agropecuarias y Ambientales",
    "AMBIENTAL": "Ciencias Agropecuarias y Ambientales",

    # --- 9. Otras Áreas (Menor Prioridad) ---
    "MILITARES": "Otras Áreas",
    "TEOLOGIA": "Otras Áreas",
    "NAVALES": "Otras Áreas",
    "NAUTICAS": "Otras Áreas",
    "DEPORTE": "Otras Áreas",
    "RECREACION": "Otras Áreas",
}

def asignar_area(programa):
    """
    Busca palabras clave del diccionario MAPEO_AREAS en el nombre
    del programa (ya normalizado) y devuelve el área correspondiente.
    """
    programa_norm = normalizar_texto(programa)

    for palabra_clave, area in MAPEO_AREAS.items():
        # se usa' in ' para buscar la palabra clave en cualquier parte del nombre
        if palabra_clave in programa_norm:
            return area

    # Si no encuentra ninguna coincidencia
    return "Área No Identificada"

zt = pd.DataFrame(zt)

# Aplica la función 'asignar_area' a cada fila de la columna 'E_PRGM_ACADEMICO'
# y guarda el resultado en la nueva columna 'F_AREA_CON_PRGM'
zt['F_AREA_CON_PRGM'] = zt['E_PRGM_ACADEMICO'].apply(asignar_area)


- La columna `RENDIMIENTO_GLOBAL` se generará utilizando la mayor parte de las demás columnas se utilizarán las siguientes y su correspondiente conversión:

  - `E_VALORMATRICULAUNIVERSIDAD`, que necesitaremos convertir a una **codificación continua**
  - `E_HORASSEMANATRABAJA`, que necesitaremos convertir a una **codificación continua**
  - `PERIODO_ACADEMICO`, que necesitaremos convertir a una **codificación onehot**
  - `E_PRGM_DEPARTAMENTO`, que necesitaremos convertir a una **codificación onehot**
  - `F_TIENEINTERNET`, que necesitaremos convertir a una **codificación onehot**
  - `F_EDUCACIONPADRE`, que necesitaremos convertir a una **codificación onehot**
  - `F_TIENELAVADORA`, que necesitaremos convertir a una **codificación onehot**
  - `F_TIENEAUTOMOVIL`, que necesitaremos convertir a una **codificación onehot**
  - `E_PAGOMATRICULAPROPIO`, que necesitaremos convertir a una **codificación onehot**
  - `F_TIENECOMPUTADOR`, que necesitaremos convertir a una **codificación onehot**
  - `F_EDUCACIONMADRE`, que necesitaremos convertir a una **codificación onehot**
  - `F_AREA_CON_PRGM`, que necesitaremos convertir a una **codificación onehot**

Se eliminarán del dataframe las siguientes columnas porque no aportan mayor información al entrenamiento del modelo:

 - `F_ESTRATOVIVIENDA`, que necesitaremos convertir a una **codificación onehot**
 - `F_TIENEINTERNET.1`
- `E_PRIVADO_LIBERTAD`, que necesitaremos convertir a una **codificación onehot**

## Limpieza Inicial

Sustitución de los valores `nan` por `'no info'` para marcarlos

In [101]:
# Sustitución de los valores `nan` con un símbolo preestablecido para marcarlos
zt.E_PRGM_DEPARTAMENTO.values[zt.E_PRGM_DEPARTAMENTO.isna()] = 'no info'
zt.E_VALORMATRICULAUNIVERSIDAD.values[zt.E_VALORMATRICULAUNIVERSIDAD.isna()] = 'no info'
zt.E_HORASSEMANATRABAJA.values[zt.E_HORASSEMANATRABAJA.isna()] = 'no info'
zt.F_ESTRATOVIVIENDA.values[zt.F_ESTRATOVIVIENDA.isna()] = 'no info'
zt.F_TIENEINTERNET.values[zt.F_TIENEINTERNET.isna()] = 'no info'
zt.F_EDUCACIONPADRE.values[zt.F_EDUCACIONPADRE.isna()] = 'no info'
zt.F_TIENELAVADORA.values[zt.F_TIENELAVADORA.isna()] = 'no info'
zt.F_TIENEAUTOMOVIL.values[zt.F_TIENEAUTOMOVIL.isna()] = 'no info'
zt.E_PRIVADO_LIBERTAD.values[zt.E_PRIVADO_LIBERTAD.isna()] = 'no info'
zt.E_PAGOMATRICULAPROPIO.values[zt.E_PAGOMATRICULAPROPIO.isna()] = 'no info'
zt.F_TIENECOMPUTADOR.values[zt.F_TIENECOMPUTADOR.isna()] = 'no info'
zt.F_EDUCACIONMADRE.values[zt.F_EDUCACIONMADRE.isna()] = 'no info'
zt.F_AREA_CON_PRGM.values[zt.F_AREA_CON_PRGM.isna()] = 'no info'

# Verificación de que ya no hay ningún valor NaN
sum(zt.E_VALORMATRICULAUNIVERSIDAD.isna()), sum(zt.E_HORASSEMANATRABAJA.isna()), sum(zt.F_ESTRATOVIVIENDA.isna()), sum(zt.F_TIENEINTERNET.isna()), sum(zt.F_EDUCACIONPADRE.isna()), sum(zt.F_TIENELAVADORA.isna()), sum(zt.F_TIENEAUTOMOVIL.isna()), sum(zt.E_PRIVADO_LIBERTAD.isna()), sum(zt.E_PAGOMATRICULAPROPIO.isna()), sum(zt.F_TIENECOMPUTADOR.isna()), sum(zt.F_EDUCACIONMADRE.isna()), sum(zt.F_AREA_CON_PRGM.isna())

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

### Conversión a valor continuo de `E_VALORMATRICULAUNIVERSIDAD`

Conversión `E_VALORMATRICULAUNIVERSIDAD` a un valor **continuo**, ya que el orden tiene sentido y se quiere que los modelos puedan capturarlo.

In [102]:
# Diccionario de phyton que mapea las categorías de texto a un número flotante que represente el punto medio del rango
cmap = {
 'No pagó matrícula': 0,
 'Menos de 500 mil': .25,
 'Entre 500 mil y menos de 1 millón': .75,
 'Entre 1 millón y menos de 2.5 millones': 1.75,
 'Entre 2.5 millones y menos de 4 millones': 3.25,
 'Entre 4 millones y menos de 5.5 millones': 4.75,
 'Entre 5.5 millones y menos de 7 millones': 6.25,
 'Más de 7 millones': 7.75,
 'no info': -1}

# Reemplaza la columna original con el array de números numpy, que se creo luego de reemplazar cada texto por el número específico
zt.E_VALORMATRICULAUNIVERSIDAD = np.r_[[cmap[i] for i in zt.E_VALORMATRICULAUNIVERSIDAD]]

# Realizar conteo de cuantas veces aparece cada valor numérico
zt.E_VALORMATRICULAUNIVERSIDAD.value_counts()

,count
E_VALORMATRICULAUNIVERSIDAD,
1.75,86923
3.25,55193
0.25,34589
0.75,33564
4.75,29855
7.75,29061
6.25,16558
0.00,8320
-1.00,1972


### Conversión a valor continuo de `E_HORASSEMANATRABAJA`

Conversión `E_HORASSEMANATRABAJA` a un valor **continuo**, ya que el orden tiene sentido y se quiere que los modelos puedan capturarlo.

In [103]:
# Diccionario de phyton que mapea las categorías de texto a un número flotante que represente el punto medio del rango
cmap = {
 '0': 0,
 'Menos de 10 horas': 5,
 'Entre 11 y 20 horas': 15.5,
 'Entre 21 y 30 horas': 25.5,
 'Más de 30 horas': 35,
 'no info': -1}

# Reemplaza la columna original con el array de números numpy, que se creo luego de reemplazar cada texto por el número específico
zt.E_HORASSEMANATRABAJA = np.r_[[cmap[i] for i in zt.E_HORASSEMANATRABAJA]]

# Realizar conteo de cuantas veces aparece cada valor numérico
zt.E_HORASSEMANATRABAJA.value_counts()

,count
E_HORASSEMANATRABAJA,
35.0,106187
0.0,50422
15.5,49385
25.5,39930
5.0,37479
-1.0,12632


### Conversión a valor continuo de `F_EDUCACIONMADRE`

Conversión `F_EDUCACIONMADRE` a un valor **continuo**, ya que el orden tiene sentido y se quiere que los modelos puedan capturarlo.

In [104]:
# Diccionario de phyton que mapea las categorías de texto a un número entero
cmap = {
 'Ninguno': 0,
 'Primaria incompleta': 1,
 'Primaria completa': 2,
 'Secundaria (Bachillerato) incompleta': 3,
 'Secundaria (Bachillerato) completa': 4,
 'Técnica o tecnológica incompleta': 5,
 'Técnica o tecnológica completa': 6,
 'Educación profesional incompleta': 7,
 'Educación profesional completa': 8,
 'Postgrado': 9,
 'no info': -1,
 'No sabe':-1,
 'No Aplica':-1 }

# Reemplaza la columna original con el array de números numpy, que se creó luego de reemplazar cada texto por el número específico
# Modificación: Remove normalizar_texto and use the cmap with initial capital letters, consistent with training data and F_EDUCACIONPADRE.
zt.loc[:, 'F_EDUCACIONMADRE'] = zt['F_EDUCACIONMADRE'].astype(str).map(cmap).fillna(-1).astype(int)

# Realizar conteo de cuantas veces aparece cada valor numérico
zt.F_EDUCACIONMADRE.value_counts()

,count
F_EDUCACIONMADRE,
4,60647
1,42147
6,38568
8,36473
3,34566
2,24396
9,20047
5,11650
-1,11559


### Conversión a valor continuo de `F_EDUCACIONPADRE`

Conversión `F_EDUCACIONPADRE` a un valor **continuo**, ya que el orden tiene sentido y se quiere que los modelos puedan capturarlo.

In [105]:
# Diccionario de phyton que mapea las categorías de texto a un número entero
cmap = {
 'Ninguno': 0,
 'Primaria incompleta': 1,
 'Primaria completa': 2,
 'Secundaria (Bachillerato) incompleta': 3,
 'Secundaria (Bachillerato) completa': 4,
 'Técnica o tecnológica incompleta': 5,
 'Técnica o tecnológica completa': 6,
 'Educación profesional incompleta': 7,
 'Educación profesional completa': 8,
 'Postgrado': 9,
 'no info': -1,
 'No sabe':-1,
 'No Aplica':-1 }

# Reemplaza la columna original con el array de números numpy, que se creó luego de reemplazar cada texto por el número específico
zt.F_EDUCACIONPADRE = np.r_[[cmap[i] for i in zt.F_EDUCACIONPADRE]]

# Realizar conteo de cuantas veces aparece cada valor numérico
zt.F_EDUCACIONPADRE.value_counts()

,count
F_EDUCACIONPADRE,
4,54835
1,53431
8,35410
3,30640
6,27044
2,24129
-1,20280
9,19282
7,11743


### `PERIODO_ACADEMICO` se parte en año y período

In [106]:
import pandas as pd

# Asegurar que la columna sea de tipo 'string'
# Esto permite cortarla de forma segura
zt.PERIODO_ACADEMICO = zt.PERIODO_ACADEMICO.astype(str)

# Extraer el Año (los primeros 4 caracteres)
zt['anio'] = zt.PERIODO_ACADEMICO.str[0:4]

# Extraer el SubPeriodo (del 4to caracter en adelante)
zt['periodo'] = zt.PERIODO_ACADEMICO.str[4:]

# Ahora se tiene dos nuevas columnas, pero AMBAS son strings.

# Se convierte el año a entero (int)
zt['anio'] = zt['anio'].astype(int)

# Se convierte el período a 'category' para que el modelo lo trate como tal
zt['periodo']  = zt['periodo'].astype('category')

print("\n--- DATOS TRANSFORMADOS ---")
print(zt)

print("\n--- NUEVOS TIPOS DE DATOS ---")
print(zt.info())


--- DATOS TRANSFORMADOS ---
            ID PERIODO_ACADEMICO                        E_PRGM_ACADEMICO  \
0       550236             20183                          TRABAJO SOCIAL   
1        98545             20203  ADMINISTRACION COMERCIAL Y DE MERCADEO   
2       499179             20212                  INGENIERIA MECATRONICA   
3       782980             20195                      CONTADURIA PUBLICA   
4       785185             20212              ADMINISTRACION DE EMPRESAS   
...        ...               ...                                     ...   
296030  496981             20195              ADMINISTRACION DE EMPRESAS   
296031  209415             20183                                 DERECHO   
296032  239074             20212                                 DERECHO   
296033  963852             20195                  INGENIERIA AERONAUTICA   
296034  792650             20212                   INGENIERIA INDUSTRIAL   

       E_PRGM_DEPARTAMENTO  E_VALORMATRICULAUNIVERSIDAD  E

### Crear el mapeo onehot para la nueva columna `periodo`

In [107]:
x = zt.periodo.values
periodo_vals = sorted(np.unique(x))
periodo_onehot_vals = {val: np.eye(len(periodo_vals))[i] for i,val in enumerate(periodo_vals)}

periodo_onehot_enc = np.r_[[periodo_onehot_vals[i] for i in zt.periodo]]

periodo_df = pd.DataFrame(periodo_onehot_enc, columns=[f"periodo__{v}" for v in periodo_onehot_vals])

## Mapeo onehot para la columna `PERIODO_ACADEMICO`

In [108]:
x = zt.PERIODO_ACADEMICO.values
PERIODO_ACADEMICO_vals = sorted(np.unique(x))
PERIODO_ACADEMICO_onehot_vals = {val: np.eye(len(PERIODO_ACADEMICO_vals))[i] for i,val in enumerate(PERIODO_ACADEMICO_vals)}
PERIODO_ACADEMICO_onehot_enc = np.r_[[PERIODO_ACADEMICO_onehot_vals[i] for i in zt.PERIODO_ACADEMICO]]
PERIODO_ACADEMICO_df = pd.DataFrame(PERIODO_ACADEMICO_onehot_enc, columns=[f"PERIODO_ACADEMICO__{v}" for v in PERIODO_ACADEMICO_onehot_vals])

### Crear el mapeo onehot de `E_PRGM_DEPARTAMENTO`

In [109]:
x = zt.E_PRGM_DEPARTAMENTO.values
E_PRGM_DEPARTAMENTO_vals = sorted(np.unique(x))
E_PRGM_DEPARTAMENTO_onehot_vals = {val: np.eye(len(E_PRGM_DEPARTAMENTO_vals))[i] for i,val in enumerate(E_PRGM_DEPARTAMENTO_vals)}
E_PRGM_DEPARTAMENTO_onehot_vals

{'AMAZONAS': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'ANTIOQUIA': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'ARAUCA': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'ATLANTICO': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'BOGOTÁ': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'BOLIVAR': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'BOYACA': array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [110]:
E_PRGM_DEPARTAMENTO_onehot_enc = np.r_[[E_PRGM_DEPARTAMENTO_onehot_vals[i] for i in zt.E_PRGM_DEPARTAMENTO]]
E_PRGM_DEPARTAMENTO_onehot_enc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Crear el mapeo onehot de `F_TIENEINTERNET`

In [111]:
x = zt.F_TIENEINTERNET.values
F_TIENEINTERNET_vals = sorted(np.unique(x))
F_TIENEINTERNET_onehot_vals = {val: np.eye(len(F_TIENEINTERNET_vals))[i] for i,val in enumerate(F_TIENEINTERNET_vals)}
F_TIENEINTERNET_onehot_vals

{'No': array([1., 0., 0.]),
 'Si': array([0., 1., 0.]),
 'no info': array([0., 0., 1.])}

In [112]:
F_TIENEINTERNET_onehot_enc = np.r_[[F_TIENEINTERNET_onehot_vals[i] for i in zt.F_TIENEINTERNET]]
F_TIENEINTERNET_onehot_enc

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [113]:
F_TIENEINTERNET_df = pd.DataFrame(F_TIENEINTERNET_onehot_enc, columns=[f"F_TIENEINTERNET__{v}" for v in F_TIENEINTERNET_onehot_vals])
F_TIENEINTERNET_df

,F_TIENEINTERNET__No,F_TIENEINTERNET__Si,F_TIENEINTERNET__no info
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0
...,...,...,...
296030,0.0,1.0,0.0
296031,0.0,1.0,0.0
296032,0.0,1.0,0.0
296033,0.0,1.0,0.0


### Crear el mapeo onehot de `E_PAGOMATRICULAPROPIO`

In [114]:
x = zt.E_PAGOMATRICULAPROPIO.values
E_PAGOMATRICULAPROPIO_vals = sorted(np.unique(x))
E_PAGOMATRICULAPROPIO_onehot_vals = {val: np.eye(len(E_PAGOMATRICULAPROPIO_vals))[i] for i,val in enumerate(E_PAGOMATRICULAPROPIO_vals)}
E_PAGOMATRICULAPROPIO_onehot_vals

{'No': array([1., 0., 0.]),
 'Si': array([0., 1., 0.]),
 'no info': array([0., 0., 1.])}

In [115]:
E_PAGOMATRICULAPROPIO_onehot_enc = np.r_[[E_PAGOMATRICULAPROPIO_onehot_vals[i] for i in zt.E_PAGOMATRICULAPROPIO]]
E_PAGOMATRICULAPROPIO_onehot_enc

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [116]:
E_PAGOMATRICULAPROPIO_df = pd.DataFrame(E_PAGOMATRICULAPROPIO_onehot_enc, columns=[f"E_PAGOMATRICULAPROPIO__{v}" for v in E_PAGOMATRICULAPROPIO_onehot_vals])
E_PAGOMATRICULAPROPIO_df

,E_PAGOMATRICULAPROPIO__No,E_PAGOMATRICULAPROPIO__Si,E_PAGOMATRICULAPROPIO__no info
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
296030,0.0,1.0,0.0
296031,1.0,0.0,0.0
296032,1.0,0.0,0.0
296033,1.0,0.0,0.0


### Crear el mapeo onehot de `F_TIENECOMPUTADOR`

In [117]:
x = zt.F_TIENECOMPUTADOR.values
F_TIENECOMPUTADOR_vals = sorted(np.unique(x))
F_TIENECOMPUTADOR_onehot_vals = {val: np.eye(len(F_TIENECOMPUTADOR_vals))[i] for i,val in enumerate(F_TIENECOMPUTADOR_vals)}
F_TIENECOMPUTADOR_onehot_vals

{'No': array([1., 0., 0.]),
 'Si': array([0., 1., 0.]),
 'no info': array([0., 0., 1.])}

In [118]:
F_TIENECOMPUTADOR_onehot_enc = np.r_[[F_TIENECOMPUTADOR_onehot_vals[i] for i in zt.F_TIENECOMPUTADOR]]
F_TIENECOMPUTADOR_onehot_enc

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [119]:
F_TIENECOMPUTADOR_df = pd.DataFrame(F_TIENECOMPUTADOR_onehot_enc, columns=[f"F_TIENECOMPUTADOR__{v}" for v in F_TIENECOMPUTADOR_onehot_vals])
F_TIENECOMPUTADOR_df

,F_TIENECOMPUTADOR__No,F_TIENECOMPUTADOR__Si,F_TIENECOMPUTADOR__no info
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0
...,...,...,...
296030,0.0,1.0,0.0
296031,0.0,1.0,0.0
296032,0.0,1.0,0.0
296033,0.0,1.0,0.0


### Crear el mapeo onehot de `F_AREA_CON_PRGM`

In [120]:
x = zt.F_AREA_CON_PRGM.values
F_AREA_CON_PRGM_vals = sorted(np.unique(x))
F_AREA_CON_PRGM_onehot_vals = {val: np.eye(len(F_AREA_CON_PRGM_vals))[i] for i,val in enumerate(F_AREA_CON_PRGM_vals)}
F_AREA_CON_PRGM_onehot_vals
F_AREA_CON_PRGM_onehot_enc = np.r_[[F_AREA_CON_PRGM_onehot_vals[i] for i in zt.F_AREA_CON_PRGM]]
F_AREA_CON_PRGM_onehot_enc
F_AREA_CON_PRGM_df = pd.DataFrame(F_AREA_CON_PRGM_onehot_enc, columns=[f"F_AREA_CON_PRGM__{v}" for v in F_AREA_CON_PRGM_onehot_vals])
F_AREA_CON_PRGM_df

,"F_AREA_CON_PRGM__Artes, Diseño y Comunicación",F_AREA_CON_PRGM__Ciencias Agropecuarias y Ambientales,F_AREA_CON_PRGM__Ciencias Básicas y Naturales,F_AREA_CON_PRGM__Ciencias Económicas y Administrativas,F_AREA_CON_PRGM__Ciencias Sociales y Humanas,F_AREA_CON_PRGM__Ciencias de la Salud,F_AREA_CON_PRGM__Educación (Licenciaturas),F_AREA_CON_PRGM__Ingeniería y Tecnología,F_AREA_CON_PRGM__Otras Áreas,F_AREA_CON_PRGM__Área No Identificada
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
296030,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
296031,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
296032,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
296033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Crear el mapeo onehot de `F_TIENELAVADORA`

In [121]:
x = zt.F_TIENELAVADORA.values
F_TIENELAVADORA_vals = sorted(np.unique(x))
F_TIENELAVADORA_onehot_vals = {val: np.eye(len(F_TIENELAVADORA_vals))[i] for i,val in enumerate(F_TIENELAVADORA_vals)}
F_TIENELAVADORA_onehot_vals
F_TIENELAVADORA_onehot_enc = np.r_[[F_TIENELAVADORA_onehot_vals[i] for i in zt.F_TIENELAVADORA]]
F_TIENELAVADORA_onehot_enc
F_TIENELAVADORA_df = pd.DataFrame(F_TIENELAVADORA_onehot_enc, columns=[f"F_TIENELAVADORA__{v}" for v in F_TIENELAVADORA_onehot_vals])


### Crear el mapeo onehot de `F_TIENEAUTOMOVIL`

In [122]:
x = zt.F_TIENEAUTOMOVIL.values
F_TIENEAUTOMOVIL_vals = sorted(np.unique(x))
F_TIENEAUTOMOVIL_onehot_vals = {val: np.eye(len(F_TIENEAUTOMOVIL_vals))[i] for i,val in enumerate(F_TIENEAUTOMOVIL_vals)}
F_TIENEAUTOMOVIL_onehot_vals
F_TIENEAUTOMOVIL_onehot_enc = np.r_[[F_TIENEAUTOMOVIL_onehot_vals[i] for i in zt.F_TIENEAUTOMOVIL]]
F_TIENEAUTOMOVIL_onehot_enc
F_TIENEAUTOMOVIL_df = pd.DataFrame(F_TIENEAUTOMOVIL_onehot_enc, columns=[f"F_TIENEAUTOMOVIL__{v}" for v in F_TIENEAUTOMOVIL_onehot_vals])
F_TIENEAUTOMOVIL_df

,F_TIENEAUTOMOVIL__No,F_TIENEAUTOMOVIL__Si,F_TIENEAUTOMOVIL__no info
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
296030,0.0,1.0,0.0
296031,1.0,0.0,0.0
296032,1.0,0.0,0.0
296033,1.0,0.0,0.0


In [123]:
print(zt.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296035 entries, 0 to 296034
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   ID                           296035 non-null  int64   
 1   PERIODO_ACADEMICO            296035 non-null  object  
 2   E_PRGM_ACADEMICO             296035 non-null  object  
 3   E_PRGM_DEPARTAMENTO          296035 non-null  object  
 4   E_VALORMATRICULAUNIVERSIDAD  296035 non-null  float64 
 5   E_HORASSEMANATRABAJA         296035 non-null  float64 
 6   F_ESTRATOVIVIENDA            296035 non-null  object  
 7   F_TIENEINTERNET              296035 non-null  object  
 8   F_EDUCACIONPADRE             296035 non-null  int64   
 9   F_TIENELAVADORA              296035 non-null  object  
 10  F_TIENEAUTOMOVIL             296035 non-null  object  
 11  E_PRIVADO_LIBERTAD           296035 non-null  object  
 12  E_PAGOMATRICULAPROPIO        296035 non-null

### Eliminación de dataframes originales y concatenación de los nuevos DataFrames 'One-hot'

Se junta todo en un solo dataframe, eliminando las columnas originales que se limpiaron y que se convirtieron en DataFrames 'One-hot'.

Se tienen columnas binarias a las que se le asigna un valor de 1 a solo una de ellas por cada fila, de acuerdo al valor asociado a cada uno de los estudiantes con el DataFrame original y sus correspondientes columnas.

In [124]:
# Lista de todas las columnas originales que se van a eliminar
columnas_originales = ['ID', 'E_PRGM_ACADEMICO','E_PRGM_DEPARTAMENTO', 'F_TIENEINTERNET','F_TIENELAVADORA','F_TIENEAUTOMOVIL',
'E_PRIVADO_LIBERTAD','E_PAGOMATRICULAPROPIO','F_TIENECOMPUTADOR','F_TIENEINTERNET.1', 'F_AREA_CON_PRGM', 'INDICADOR_3', 'INDICADOR_4', 'F_ESTRATOVIVIENDA']

# Lista de TODOS los DataFrames que se van a unir
dataframes_para_unir = [
    zt,
     F_TIENEINTERNET_df, F_TIENECOMPUTADOR_df, F_AREA_CON_PRGM_df, F_TIENELAVADORA_df, F_TIENEAUTOMOVIL_df, E_PAGOMATRICULAPROPIO_df
    ]

# Concatena todo: zt + todos los nuevos DataFrames
zt_final = pd.concat(dataframes_para_unir, axis=1)

# Elimina todas las columnas originales
zt_final = zt_final.drop(columns=columnas_originales)

# Revisar el resultado de la concatenación
print(zt_final.shape)
zt_final

(296035, 34)


,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_EDUCACIONPADRE,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,anio,periodo,F_TIENEINTERNET__No,...,F_AREA_CON_PRGM__Área No Identificada,F_TIENELAVADORA__No,F_TIENELAVADORA__Si,F_TIENELAVADORA__no info,F_TIENEAUTOMOVIL__No,F_TIENEAUTOMOVIL__Si,F_TIENEAUTOMOVIL__no info,E_PAGOMATRICULAPROPIO__No,E_PAGOMATRICULAPROPIO__Si,E_PAGOMATRICULAPROPIO__no info
0,20183,0.25,5.0,6,2,0.328,0.219,2018,3,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,20203,3.25,25.5,4,6,0.227,0.283,2020,3,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,20212,1.75,0.0,3,4,0.285,0.228,2021,2,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,20195,1.75,25.5,1,1,0.160,0.408,2019,5,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,20212,3.25,15.5,4,4,0.209,0.283,2021,2,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296030,20195,3.25,35.0,1,1,0.168,0.410,2019,5,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
296031,20183,1.75,0.0,8,8,0.471,0.184,2018,3,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
296032,20212,3.25,35.0,4,8,0.292,0.249,2021,2,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
296033,20195,6.25,15.5,8,8,0.305,0.219,2019,5,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [125]:
zt_final = zt_final.drop(columns=['periodo'])

In [126]:
print(zt_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296035 entries, 0 to 296034
Data columns (total 33 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   PERIODO_ACADEMICO                                       296035 non-null  object 
 1   E_VALORMATRICULAUNIVERSIDAD                             296035 non-null  float64
 2   E_HORASSEMANATRABAJA                                    296035 non-null  float64
 3   F_EDUCACIONPADRE                                        296035 non-null  int64  
 4   F_EDUCACIONMADRE                                        296035 non-null  object 
 5   INDICADOR_1                                             296035 non-null  float64
 6   INDICADOR_2                                             296035 non-null  float64
 7   anio                                                    296035 non-null  int64  
 8   F_TIENEINTERNET__No     

In [127]:
# Guardar los id de los estudiantes
zt_ids = zt['ID'].values

# Se sobreescribe el DataFrame zt unicamente con las columnas especificadas
zt = zt[['PERIODO_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE', 'F_AREA_CON_PRGM','INDICADOR_1', 'INDICADOR_2']]

# Revisar dimensiones de zt
print ("Dimensiones del DataFrame cargado", zt.shape)

# Mostrar las primeras 5 filas del nuevo dataframe
zt.head()

Dimensiones del DataFrame cargado (296035, 14)


,PERIODO_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,F_AREA_CON_PRGM,INDICADOR_1,INDICADOR_2
0,20183,BOLIVAR,0.25,5.0,Si,6,Si,No,Si,Si,2,Ciencias Sociales y Humanas,0.328,0.219
1,20203,ANTIOQUIA,3.25,25.5,Si,4,Si,No,No,Si,6,Ciencias Económicas y Administrativas,0.227,0.283
2,20212,BOGOTÁ,1.75,0.0,Si,3,Si,No,No,Si,4,Ingeniería y Tecnología,0.285,0.228
3,20195,SUCRE,1.75,25.5,No,1,Si,No,No,No,1,Ciencias Económicas y Administrativas,0.160,0.408
4,20212,ATLANTICO,3.25,15.5,Si,4,Si,No,No,Si,4,Ciencias Económicas y Administrativas,0.209,0.283


create `X` array and predict

In [128]:
X_test_data = zt_final[sorted(zt_final.columns)]
X_test_data.shape

(296035, 33)

In [129]:
print(zt_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296035 entries, 0 to 296034
Data columns (total 33 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   PERIODO_ACADEMICO                                       296035 non-null  object 
 1   E_VALORMATRICULAUNIVERSIDAD                             296035 non-null  float64
 2   E_HORASSEMANATRABAJA                                    296035 non-null  float64
 3   F_EDUCACIONPADRE                                        296035 non-null  int64  
 4   F_EDUCACIONMADRE                                        296035 non-null  object 
 5   INDICADOR_1                                             296035 non-null  float64
 6   INDICADOR_2                                             296035 non-null  float64
 7   anio                                                    296035 non-null  int64  
 8   F_TIENEINTERNET__No     

In [130]:
preds_test_data = rf.predict(X_test_data)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


## Preparar la entrega

Primero, convierte las predicciones de nuevo a sus valores textuales.

In [131]:
# Mapeo inverso
rmapi = {v:k for k,v in rmap.items()}
text_preds_test_data = [rmapi[i] for i in preds_test_data]


In [132]:
# crear dataframe

submission = pd.DataFrame([zt_ids, text_preds_test_data], index=['ID', 'RENDIMIENTO_GLOBAL']).T
submission

,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo
...,...,...
296030,496981,bajo
296031,209415,alto
296032,239074,alto
296033,963852,alto


In [133]:
# Guardar archivo listo para entregar
submission.to_csv("my_submission.csv", index=False)

In [134]:
!head my_submission.csv

ID,RENDIMIENTO_GLOBAL
550236,alto
98545,medio-alto
499179,alto
782980,bajo
785185,bajo
58495,medio-bajo
705444,alto
557548,alto
519909,medio-alto


In [135]:
submission.shape

(296035, 2)

## Send your submission to Kaggle

You must **join the competition** first

In [ ]:
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f my_submission.csv -m "Entrega actualizada por Pablo Uribe"

100% 4.04M/4.04M [00:00<00:00, 5.48MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia